In [7]:
#Import dependants
import pandas as pd
from pathlib import Path
import requests
import json
from pprint import pprint

from api_keys import geoapify_key

In [14]:
# load and read the CSVs
Kaggle1_path = Path("Resources/Kaggle1DS.csv")
Kaggle1_df = pd.read_csv(Kaggle1_path)

Kaggle1_df.head()

,LONCOD,LATCOD,CDCODE,year,school,city,state,address,urbanrural,race,killed,injured,victims,date,type,NCESSCH,time
0,-75.584900,39.668100,1000.0,2009,William Penn High School,New Castle,Delaware,713 E Basin Rd,suburbtown,BLACK,0,1,1,8-Jan-09,High School,1.000230e+11,8:30PM
1,-87.617500,41.839900,1701.0,2009,Dunbar Vocational Career Academy,Chicago,Illinois,3000 S King Dr,urban,BLACK,0,5,5,9-Jan-09,High School,1.709930e+11,8:00PM
2,-90.165100,38.565700,1712.0,2009,Cahokia High School,Cahokia,Illinois,800 Range Ln,suburbtown,BLACK,1,0,1,23-Jan-09,High School,1.708040e+11,7:20PM
3,-78.311800,35.831500,3713.0,2009,Zebulon Middle School,Zebulon,North Carolina,Halifax Road and New Oak Street,suburbtown,BLACK,0,1,1,11-Feb-09,Middle School,3.704720e+11,6:00AM
4,-83.113977,42.383593,NaN,2009,Central High School,Detroit,Michigan,2425 Tuxedo St,urban,NaN,0,1,1,17-Feb-09,High School,2.601103e+11,2:20PM


In [15]:
# Eliminate current "Longitude" and "Latitude" to make way for the new ones
Kaggle1_dropped = Kaggle1_df.drop(columns=["LONCOD", "LATCOD"], axis=1)
Kaggle1_dropped.head()

,CDCODE,year,school,city,state,address,urbanrural,race,killed,injured,victims,date,type,NCESSCH,time
0,1000.0,2009,William Penn High School,New Castle,Delaware,713 E Basin Rd,suburbtown,BLACK,0,1,1,8-Jan-09,High School,1.000230e+11,8:30PM
1,1701.0,2009,Dunbar Vocational Career Academy,Chicago,Illinois,3000 S King Dr,urban,BLACK,0,5,5,9-Jan-09,High School,1.709930e+11,8:00PM
2,1712.0,2009,Cahokia High School,Cahokia,Illinois,800 Range Ln,suburbtown,BLACK,1,0,1,23-Jan-09,High School,1.708040e+11,7:20PM
3,3713.0,2009,Zebulon Middle School,Zebulon,North Carolina,Halifax Road and New Oak Street,suburbtown,BLACK,0,1,1,11-Feb-09,Middle School,3.704720e+11,6:00AM
4,NaN,2009,Central High School,Detroit,Michigan,2425 Tuxedo St,urban,NaN,0,1,1,17-Feb-09,High School,2.601103e+11,2:20PM


In [16]:
# Function to geocode a city and state and update the DataFrame
def geocode_and_update(city, state, api_key):
    query = f"{city}, {state}"
    url = f"https://api.geoapify.com/v1/geocode/search?text={query}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
    return None

# Example usage: Loop through cities and states in the DataFrame and geocode
api_key = geoapify_key
Kaggle1_dropped['Latitude'] = None  # Create new columns for latitude and longitude
Kaggle1_dropped['Longitude'] = None
for index, row in Kaggle1_df.iterrows():
    city = row['city']
    state = row['state']
    coordinates = geocode_and_update(city, state, api_key)
    if coordinates:
        Kaggle1_dropped.at[index, 'Latitude'] = coordinates[1]  # Assuming Geoapify returns latitude first
        Kaggle1_dropped.at[index, 'Longitude'] = coordinates[0]
        print(f"Geocoded {city}, {state}")
    else:
        print(f"Failed to geocode {city}, {state}")

# View DataFrame with "Latitude" and "Longitude" re-added
Kaggle1_dropped.head()        

Geocoded New Castle, Delaware
Geocoded Chicago, Illinois
Geocoded Cahokia, Illinois
Geocoded Zebulon, North Carolina
Geocoded Detroit, Michigan
Geocoded Fayetteville, North Carolina
Geocoded Richmond, Virginia
Geocoded Lexington, Kentucky
Geocoded Parkersburg, Iowa
Geocoded Mattituck, New York
Geocoded Long Beach, California
Geocoded Brockton, Massachusetts
Geocoded Shreveport, Louisiana
Geocoded Livingston, Alabama
Geocoded Madison, Alabama
Geocoded Knoxville, Tennessee
Geocoded Littleton, Colorado
Geocoded Tacoma, Washington
Geocoded South Gate, California
Geocoded Salinas, California
Geocoded Carlsbad, California
Geocoded Omaha, Nebraska
Geocoded Gardena, California
Geocoded Placerville, California
Geocoded Martinsville, Indiana
Geocoded Houston, Texas
Geocoded Apopka, Florida
Geocoded Houston, Texas
Geocoded Pearl City, Hawaii
Geocoded Everett, Washington
Geocoded Harper Woods, Michigan
Geocoded Fayetteville, North Carolina
Geocoded Edinburg, Texas
Geocoded Houston, Texas
Geocoded 

,CDCODE,year,school,city,state,address,urbanrural,race,killed,injured,victims,date,type,NCESSCH,time,Latitude,Longitude
0,1000.0,2009,William Penn High School,New Castle,Delaware,713 E Basin Rd,suburbtown,BLACK,0,1,1,8-Jan-09,High School,1.000230e+11,8:30PM,39.662057,-75.566313
1,1701.0,2009,Dunbar Vocational Career Academy,Chicago,Illinois,3000 S King Dr,urban,BLACK,0,5,5,9-Jan-09,High School,1.709930e+11,8:00PM,41.875562,-87.624421
2,1712.0,2009,Cahokia High School,Cahokia,Illinois,800 Range Ln,suburbtown,BLACK,1,0,1,23-Jan-09,High School,1.708040e+11,7:20PM,38.56616,-90.179424
3,3713.0,2009,Zebulon Middle School,Zebulon,North Carolina,Halifax Road and New Oak Street,suburbtown,BLACK,0,1,1,11-Feb-09,Middle School,3.704720e+11,6:00AM,35.822162,-78.313213
4,NaN,2009,Central High School,Detroit,Michigan,2425 Tuxedo St,urban,NaN,0,1,1,17-Feb-09,High School,2.601103e+11,2:20PM,42.331551,-83.04664


In [17]:
# Save the updated DataFrame to a new CSV file
Kaggle1_dropped.to_csv('Outputs/clean_1_updated.csv', index=False)